In [1]:
!pip install torch transformers accelerate bitsandbytes unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 908.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 152.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0

In [2]:
# Cargar modelo y tokenizador
from transformers import AutoModel, AutoTokenizer
from unsloth import FastLanguageModel
import torch

model_name = "AdrianML7/asesor-finanzas-phi3"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, #Repo publico
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/tmp/ipython-input-2316991691.py:3: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.5: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/478M [00:00<?, ?B/s]

Unsloth 2025.12.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32009)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj

In [3]:
#Nuestro dataset tiene instruccion, input y output
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. The output must have a clean structure
separated by paragraphs.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

instruction = input("Cual es tu pregunta? ")
input_context = input("Cual es tu perfil? ")

Cual es tu pregunta? En qué activos financieros puedo invertir?
Cual es tu perfil? joven con ahorros de 1000 euros mensuales


In [4]:
#Preparar prompt

#Adapta el prompt y tokeniza
prompt = alpaca_prompt.format(instruction, input_context, "")
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

In [5]:
#Prepara la respuesta y muestrala por pantalla
from IPython.display import Markdown, display

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
texto_crudo = tokenizer.batch_decode(outputs)[0]

# Limpiamos para quedarnos solo con la respuesta del asistente
respuesta_limpia = texto_crudo.split("### Response:\n")[-1].replace("<|endoftext|>", "")

# Renderizar Markdown
display(Markdown(respuesta_limpia))

¡Hola! Es genial que quieras dar tus primeros pasos en la inversión. Con 1.000 € al mes que dispones, puedes construir una base sólida y diversificada.

**1. Fondo de emergencia**
Antes de invertir, destina entre 3 y 6 meses de tus gastos fijos a una cuenta de alta liquidez (cuenta de ahorros o depósito a corto plazo). Esto te protege ante imprevistos y evita que tengas que vender inversiones en momentos desfavorables.

**2. Plan de pensiones o fondos de inversión a largo plazo**
Si tu objetivo es la jubilación, los planes de pensiones o fondos indexados de bajo coste son una opción eficaz. Con una inversión mensual de 1.000 €, podrías acumular un capital significativo tras 30‑35 años, gracias al efecto del interés compuesto.

**3. Fondos indexados de bajo coste**
Para una exposición diversificada a bajo riesgo, los fondos que replican índices globales (por ejemplo, MSCI World o S&P 500) son recomendables. Su gestión pasiva reduce comisiones y su amplio portafolio reduce la volatilidad.

**4. Bonos o fondos de renta fija**
Si prefieres una menor volatilidad, destina una parte (aprox. 20‑30 %) a bonos gubernamentales o fondos de renta fija. Estos instrumentos suelen ofrecer rendimientos estables y preservan el capital.

**5. Inversión en acciones de alta calidad**
Con el resto de tu capital, puedes destinar entre 30‑40 % a acciones de empresas consolidadas con dividendos. Una cartera diversificada internacional reduce el riesgo específico de una empresa.

**6. Educación continua**
Dedica tiempo a aprender conceptos básicos (valoración, diversificación, horizonte temporal) y revisa tu estrategia al menos una vez al año.

**Advertencia de riesgo**
Todas las inversiones conllevan riesgo de pérdida de capital. Los fondos indexados y bonos están sujetos a la evolución de los tipos de interés y a la inflación, mientras que las acciones pueden experimentar fluctuaciones significativas. Nunca inviertas dinero que no puedas permitirte perder y considera consultar a un asesor financiero certificado antes de tomar decisiones importantes.

Con estos pasos podrás iniciar una cartera diversificada y alineada a tu perfil de riesgo, mientras construyes una base financiera sólida para el futuro.